In [3]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

%matplotlib inline

In [4]:
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = 'Documents/ECE228_Project/stage1_train/'
TEST_PATH = 'Documents/ECE228_Project/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [5]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [4]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

Getting and resizing train images and masks ... 


100%|██████████| 670/670 [01:54<00:00,  5.83it/s]

Getting and resizing test images ... 



100%|██████████| 65/65 [00:00<00:00, 76.13it/s]

Done!


In [61]:
np.save('Xvariables.npy',X_train)
np.save('Yvariables.npy',Y_train)

In [6]:
X_train=np.load('Xvariables.npy')
Y_train=np.load('Yvariables.npy').astype(np.uint8)



In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv6 = nn.Conv2d(64, 64, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.conv7 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv8 = nn.Conv2d(128, 128, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(2,2)
        
        self.conv9 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(256)
        
        self.convtran1 = nn.ConvTranspose2d(256,128,2,stride=2)
        
        self.conv11 = nn.Conv2d(256,128,3, padding=1)
        self.conv12 = nn.Conv2d(128, 128,3, padding=1)
        self.batchnorm6 = nn.BatchNorm2d(128)
        
        self.convtran2 = nn.ConvTranspose2d(128,64,2,stride=2)
        
        self.conv13 = nn.Conv2d(128,64,3, padding=1)
        self.conv14 = nn.Conv2d(64,64,3, padding=1)
        self.batchnorm7 = nn.BatchNorm2d(64)
        
        self.convtran3 = nn.ConvTranspose2d(64,32,2,stride=2)
        
        self.conv15 = nn.Conv2d(64,32,3, padding=1)
        self.conv16 = nn.Conv2d(32,32,3, padding=1)
        self.batchnorm8 = nn.BatchNorm2d(32)
        
        self.convtran4 = nn.ConvTranspose2d(32,16,2,stride=2)
        
        self.conv17 = nn.Conv2d(32,16,3, padding=1)
        self.conv18 = nn.Conv2d(16, 16,3, padding=1)
        self.batchnorm9 = nn.BatchNorm2d(16)
        
        self.conv19 = nn.Conv2d(16,1,1)

    def forward(self, x):
        c1 = F.relu(self.batchnorm1(self.conv1(x)))
        c1 = F.relu(self.batchnorm1(self.conv2(c1)))
        p1 = self.pool1(c1)
        
        c2 = F.relu(self.batchnorm2(self.conv3(p1)))
        c2 = F.relu(self.batchnorm2(self.conv4(c2)))
        p2 = self.pool2(c2)
        
        c3 = F.relu(self.batchnorm3(self.conv5(p2)))
        c3 = F.relu(self.batchnorm3(self.conv6(c3)))
        p3 = self.pool3(c3)
        
        c4 = F.relu(self.batchnorm4(self.conv7(p3)))
        c4 = F.relu(self.batchnorm4(self.conv8(c4)))
        p4 = self.pool4(c4)
        
        c5 = F.relu(self.batchnorm5(self.conv9(p4)))
        c5 = F.relu(self.batchnorm5(self.conv10(c5)))
        
        u6 = self.convtran1(c5)
        u6 = torch.cat((u6,c4),dim=1)
        
        c6 = F.relu(self.batchnorm6(self.conv11(u6)))
        c6 = F.relu(self.batchnorm6(self.conv12(c6)))
        
        u7 = self.convtran2(c6)
        u7 = torch.cat((u7,c3),dim=1)
        
        c7 = F.relu(self.batchnorm7(self.conv13(u7)))
        c7 = F.relu(self.batchnorm7(self.conv14(c7)))
        
        u8 = self.convtran3(c7)
        u8 = torch.cat((u8,c2),dim=1)
        
        c8 = F.relu(self.batchnorm8(self.conv15(u8)))
        c8 = F.relu(self.batchnorm8(self.conv16(c8)))
        
        u9 = self.convtran4(c8)
        u9 = torch.cat((u9,c1),dim=1)
        
        c9 = F.relu(self.batchnorm9(self.conv17(u9)))
        c9 = F.relu(self.batchnorm9(self.conv18(c9)))
        
        out = F.sigmoid(self.conv19(c9))
        
        return out

net = Net()

In [8]:
criterion = nn.BCELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(),lr=0.01)

X_train = X_train/127 - 1


In [ ]:
np.random.seed
max_epochs=200
batchSize=1
numWorkers=1
Loss = np.zeros(max_epochs)

num_train = len(X_train)
indices = list(range(num_train))
split = int(np.floor(0.1*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

Inputs = Variable(torch.FloatTensor(np.moveaxis(X_train,3,1)))
Targets = Variable(torch.FloatTensor(np.moveaxis(Y_train,3,1)))
iteration = 0
for epoch in range(max_epochs):
    
    np.random.shuffle(train_idx)
    
    running_loss = 0
    V_loss = 0
    
    print("epoch: ",epoch)
    
    for i in range(len(train_idx)/batchSize):
        print(iteration)
        iteration+=1
        optimizer.zero_grad()
        
        inputs = Inputs[train_idx[i*batchSize:(i+1)*batchSize]]
        targets = Targets[train_idx[i*batchSize:(i+1)*batchSize]]
        outputs = net(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss   
    
    plt.imshow(np.squeeze(Y_train[train_idx[i*batchSize]]))
    plt.title('target')
    plt.show()

    plt.imshow(np.squeeze(outputs[0].data.cpu().numpy()))
    plt.title('output')
    plt.show()
    
    Loss[epoch] = running_loss/(len(train_idx)/batchSize)    
    print('running loss: ', running_loss)
    
    for i in valid_idx:
        Var=np.expand_dims(X_train[i],axis=0)
        Y=np.expand_dims(Y_train[i],axis=0)
        V_output = net(Variable(torch.FloatTensor(np.moveaxis(Var,3,1))))        
        V_loss = criterion(V_output,Variable(torch.Tensor(np.moveaxis(Y,3,1))))
                

('epoch: ', 0)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273